In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2024.4.0 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.6.4 requires scipy<1.12,>=1.4.1, but you have scipy 1.13.0 which is incompatible.


In [4]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"

In [5]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()

2024-04-10 04:47:57.915035: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 04:47:57.915195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 04:47:58.184268: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ·····································
Add token as git credential? (Y/n)  Y


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

<a name='2'></a>
#### 2. Loading dataset

In [7]:
# https://huggingface.co/datasets/neil-code/dialogsum-test
huggingface_dataset_name = "AgamP/techshila_ml"
dataset = load_dataset(huggingface_dataset_name)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Job Position', 'Question', 'Answer', 'Interview Phase', 'Answer Quality'],
        num_rows: 2323
    })
})

In [8]:
dataset=dataset['train'].train_test_split(test_size=0.3)

This is what the data looks like:

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Job Position', 'Question', 'Answer', 'Interview Phase', 'Answer Quality'],
        num_rows: 1626
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Job Position', 'Question', 'Answer', 'Interview Phase', 'Answer Quality'],
        num_rows: 697
    })
})

In [45]:
dataset['train'][0]

{'Unnamed: 0': 2037,
 'Job Position': 'Nurse',
 'Question': 'what would you do if a patient complains about their pain not being managed effectively',
 'Answer': "The primary step is to empathetically listen to the patient and affirm their experience. A comprehensive pain evaluation would be performed to comprehend the intensity and nature of their discomfort. This information would be shared with the patient's medical team to revisit and potentially revise their pain control plan. It's crucial to keep the patient informed about any modifications and engage them in the decisionmaking process fostering a sense of mutual trust and control.",
 'Interview Phase': 'Communication',
 'Answer Quality': 'Unrated'}

<a name='3'></a>
#### 3. Create bitsandbytes configuration

**To load the model, we need a configuration class that specifies how we want the quantization to be performed. We’ll achieve this with BitesAndBytesConfig from the Transformers library. This will allow us to load our LLM in 4 bits. This way, we can divide the used memory by 4 and import the model on smaller devices.**

In [11]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )
device_map = {"": 0}

<a name='4'></a>
#### 4. Load Base Model
Let's now load Phi-2 using 4-bit quantization!

In [12]:
!pip install accelerate

In [13]:
model_name='microsoft/phi-2'
original_model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

<a name='5'></a>
#### 5. Tokenization
Set up the tokenizer. Add padding on the left as it [makes training use less memory](https://ai.stackexchange.com/questions/41485/while-fine-tuning-a-decoder-only-llm-like-llama-on-chat-dataset-what-kind-of-pa).

In [14]:
# https://ai.stackexchange.com/questions/41485/while-fine-tuning-a-decoder-only-llm-like-llama-on-chat-dataset-what-kind-of-pa
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_name, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=1000, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.1,num_beams=1,top_p=0.95,).to('cpu')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<a name='6'></a>
#### 6. Test the Model with Zero Shot Inferencing

In [16]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 10

prompt = dataset['train'][index]['Question']
summary = dataset['train'][index]['Answer']
job_role=dataset['train'][index]['Job Position']
interview_phase=dataset['train'][index]['Interview Phase']

formatted_prompt = f"Instruct: You are an excellent interviewer for the given job role {job_role} and your task is to ask relevant questions to the candidate similar to {prompt}\nOutput:\n"
res = gen(original_model,formatted_prompt,100,)
#print(res[0])
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'Answer:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: You are an excellent interviewer for the given job role Nurse and your task is to ask relevant questions to the candidate similar to how do you stay up to date on medical advancements and procedures
Output:

---------------------------------------------------------------------------------------------------
Answer:
I stay current by reading medical journals attending conferences and participating in online learning modules. I also network with colleagues and discuss new developments with physicians and nurses.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
1. How do you stay up to date on the latest medical advancements and procedures?
2. Can you describe a time when you had to quickly learn a new medical procedure?
3. How do you ensure that you are providing the best possible care 

<a name='7'></a>
#### 7. Pre-processing dataset

In [17]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters 
    :param sample: Sample dictionnary
    """
    intro_prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: You are an excellent interviewer for Job Roles mentioned by the candidate, Your task is to evaluate the answers by the candidate and generate questions relevant to the Job Role while keeping in mind the interview phase i.e. General, Technical, Behavioural,etc"
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"
    
    intro = f"\n{intro_prompt}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['Job Position']}"+f"{sample['Interview Phase']}"+f"{sample['Question']}" if sample["Question"] else None
    response = f"{RESPONSE_KEY}\n{sample['Answer']}"
    end = f"{END_KEY}"
    
    parts = [part for part in [intro, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

In [18]:
# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

In [21]:
from functools import partial

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """
    
    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#, batched=True)
    
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        #remove_columns=['id', 'topic', 'dialogue', 'summary'],
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    
    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [22]:
# ## Pre-process dataset
max_length = get_max_length(original_model)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset['test'])

Found max lenth: 2048
2048
Preprocessing dataset...


Map:   0%|          | 0/1626 [00:00<?, ? examples/s]

Map:   0%|          | 0/1626 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1626 [00:00<?, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/697 [00:00<?, ? examples/s]

Map:   0%|          | 0/697 [00:00<?, ? examples/s]

Filter:   0%|          | 0/697 [00:00<?, ? examples/s]

In [23]:
print(f"Shapes of the datasets:")
print(f"Training: {train_dataset.shape}")
print(f"Validation: {eval_dataset.shape}")
#print(train_dataset)

Shapes of the datasets:
Training: (1626, 9)
Validation: (697, 9)


<a name='8'></a>
#### 8. Setup the PEFT/LoRA model for Fine-Tuning

In [24]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 262364160
all model parameters: 1521392640
percentage of trainable model parameters: 17.24%


In [25]:
print(original_model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layern

In [26]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

# 2 - Using the prepare_model_for_kbit_training method from PEFT
original_model = prepare_model_for_kbit_training(original_model)

peft_model = get_peft_model(original_model, config)

Once everything is set up and the base model is prepared, we can use the print_trainable_parameters() helper function to see how many trainable parameters are in the model.

In [27]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 20971520
all model parameters: 1542364160
percentage of trainable model parameters: 1.36%


In [28]:
print(peft_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4

<a name='9'></a>
#### 9. Train PEFT Adapter

Define training arguments and create Trainer instance.

In [31]:
output_dir = './peft-dialogue-summary-training/final-checkpoint'
import transformers

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=5,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=1,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=1,
    evaluation_strategy="steps",
    eval_steps=1,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [32]:
peft_training_args.device

device(type='cuda', index=0)

In [33]:
peft_trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
1,2.191000,2.935906
2,2.445700,2.847509
3,2.629500,2.763322
4,2.564500,2.710619
5,2.647700,2.679134


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain curr

TrainOutput(global_step=5, training_loss=2.495686435699463, metrics={'train_runtime': 706.4528, 'train_samples_per_second': 0.057, 'train_steps_per_second': 0.007, 'total_flos': 182343221821440.0, 'train_loss': 2.495686435699463, 'epoch': 0.02})

In [ ]:
print_gpu_utilization()

Saving the trained adapter

In [ ]:
import os 
os.chdir(r'/kaggle/working')
from IPython.display import FileLink 
FileLink(r'peft-dialogue-summary-training')

Pushing model on huggingface repo

In [42]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ·····································
Add token as git credential? (Y/n)  Y


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [44]:
from huggingface_hub import create_repo
create_repo("phi-2-PEFT-e5")

RepoUrl('https://huggingface.co/AgamP/phi-2-PEFT-e5', endpoint='https://huggingface.co', repo_type='model', repo_id='AgamP/phi-2-PEFT-e5')

In [48]:
from huggingface_hub import HfApi

api=HfApi()

api.upload_folder(
    folder_path="/kaggle/working/peft-dialogue-summary-training/final-checkpoint/checkpoint-5",
    repo_id="AgamP/phi-2-PEFT-e5",
    repo_type="model",
)

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/42.3M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AgamP/phi-2-PEFT-e5/commit/e73e4a074da18c3872e44cd0dc852a159119af0d', commit_message='Upload folder using huggingface_hub', commit_description='', oid='e73e4a074da18c3872e44cd0dc852a159119af0d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Free memory for merging weights
del original_model
del peft_trainer
torch.cuda.empty_cache()

In [ ]:
print_gpu_utilization()

<a name='10'></a>
#### 10. Evaluate the Model Qualitatively (Human Evaluation)

In [50]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, 
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [51]:
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [52]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/kaggle/working/peft-dialogue-summary-training/final-checkpoint/checkpoint-5",torch_dtype=torch.float16,is_trainable=False)

In [54]:
%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 1000

prompt = dataset['train'][index]['Question']
summary = dataset['train'][index]['Answer']
job_role=dataset['train'][index]['Job Position']
interview_phase=dataset['train'][index]['Interview Phase']

formatted_prompt = f"Instruct: You are an excellent interviewer for the given job role {job_role} and your task is to ask relevant questions to the candidate similar to {prompt}\nOutput:\n"
res = gen(original_model,formatted_prompt,100,)
#print(res[0])
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'Answer:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 11.9 µs
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: You are an excellent interviewer for the given job role Data Scientist and your task is to ask relevant questions to the candidate similar to Describe your experience with model explainability techniques and their importance in machine learning projects.
Output:

---------------------------------------------------------------------------------------------------
Answer:
I have experience with model explainability techniques such as feature importance analysis, partial dependence plots, SHAP values, and LIME (Local Interpretable Model-agnostic Explanations). These techniques help understand the contribution of features to model predictions, identify patterns and correlations, and explain the decision-making process of complex machine learning models to stakeholders. Model explainability enhances transpa

<a name='11'></a>
#### 11. Evaluate the Model Quantitatively (with ROUGE Metric)
Perform inferences for the sample of the test dataset (only 10 dialogues and summaries to save time). 

In [55]:
original_model = AutoModelForCausalLM.from_pretrained(base_model_id, 
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset[]

In [81]:
df1=pd.read_csv("/kaggle/input/combined-data-techshila/data (1).csv")
df1.head()

,Unnamed: 0,Job Position,Question,Answer,Interview Phase,Answer Quality
0,0,Customer Service Representative,What are Your Biggest Achievements?,"During my last job, I¬†learned¬†some interpers...",General,Average
1,1,Customer Service Representative,Name any Two Improvements You Made in the Prev...,"As a few of my team members were late to work,...",General,Good
2,2,Customer Service Representative,Tell me about a professional accomplishment yo...,One of my proudest professional accomplishment...,General,Good
3,3,Customer Service Representative,Have you ever utilized customer feedback to en...,"Yes, I've leveraged customer feedback to impro...",Role Specific,Average
4,4,Customer Service Representative,Have You Used Customer Feedback to Ensure Busi...,"Yes, I have used customer feedback to improve ...",Role Specific,Good


In [74]:
questions = dataset['test'][595:600]['Question']
job_position=dataset['test'][595:600]['Job Position']
interview_phase=dataset['test'][595:600]['Interview Phase']
answer_quality=dataset['test'][595:600]['Answer Quality']
answer = dataset['test'][595:600]['Answer']

In [75]:
questions

['why did you become a nursing assistant',
 'what would you do if you had a difficult patient',
 'how would you deal with a rude patient',
 'tell us about a time when you had to do something difficult how did you handle that and how can that help you as a Certified nursing assistants',
 'Can you discuss the challenges of working with big data and techniques to handle scalability, performance, and resource constraints?']

In [85]:
ground_truth_questions=df1.loc[df1['Job Position'].isin(job_position),['Question','Job Position']]
ground_truth_questions

,Question,Job Position
790,why did you decide on a career as a nurse,Nurse
791,why did you decide on a career as a nurse,Nurse
792,why did you decide on a career as a nurse,Nurse
793,why did you decide on a career as a nurse,Nurse
794,why did you decide on a career as a nurse,Nurse
...,...,...
2247,Describe your experience with deep learning ar...,Data Scientist
2248,How do you approach building interpretable mac...,Data Scientist
2249,Can you explain the concept of feature selecti...,Data Scientist
2250,Describe your experience with Bayesian statist...,Data Scientist


In [90]:
ground_truth1=ground_truth_questions.loc[ground_truth_questions['Job Position']==job_position[1],'Question']
ground_truth1

790             why did you decide on a career as a nurse
791             why did you decide on a career as a nurse
792             why did you decide on a career as a nurse
793             why did you decide on a career as a nurse
794             why did you decide on a career as a nurse
                              ...                        
2047    what techniques would you use to calm an upset...
2048    can you recall an episode at work where you ma...
2049    in what ways has your prior work history equip...
2050    can you describe a situation where you had to ...
2051    how would you handle a situation where you dis...
Name: Question, Length: 653, dtype: object

In [87]:
import pandas as pd

original_model_response = []
ground_truth_response = []
peft_model_response = []



for idx, question in enumerate(questions):
    #human_baseline_text_output = human_baseline_summaries[idx]
    ground_truth=ground_truth_questions.loc[ground_truth_questions['Job Position']==job_position[idx],'Question']
    prompt = f"Instruct: You are an excellent interviewer for the given job role {job_position[idx]} and your task is to ask relevant questions to the candidate like {questions[idx]} but not exactly same.You have to also rate the answer of the candidate like {answer[idx]} like {answer_quality[idx]} and give a summary \nOutput:\n"
    
    original_model_res = gen(original_model,prompt,500,)
    original_model_text_output = original_model_res[0].split('Output:')[1]
    
    peft_model_res = gen(ft_model,prompt,100,)
    peft_model_output = peft_model_res[0].split('Output:')[1]
    print(peft_model_output)
    peft_model_text_output, success, result = peft_model_output.partition('#End')
    

    original_model_response.append(original_model_text_output)
    peft_model_response.append(peft_model_text_output)
    ground_truth_response.append(ground_truth)

zipped_response = list(zip(ground_truth_response,original_model_response, peft_model_response))
 
df = pd.DataFrame(zipped_response, columns = ['ground_truth_response','original_model_response', 'peft_model_response'])
df

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



1. Can you tell me about your experience as a nursing assistant?
2. What motivated you to become a nursing assistant?
3. How do you handle difficult situations with patients or colleagues?
4. Can you give an example of a time when you went above and beyond to provide quality care for a patient?
5. How do you stay up-to-date with the latest medical advancements and practices?
6. How do you prioritize tasks and manage your time effectively as a


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



1. Can you tell me about a time when you had to handle a difficult patient?
2. How did you manage the situation?
3. What steps did you take to ensure that patient care remained the top priority?
4. Did you seek guidance from supervisors or colleagues?
5. How did you address the patient's concerns?
6. How did you handle any emotional escalation during the situation?
7. Can you give an example of a time when you had to handle


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



1. Can you tell me about a time when you had to do something difficult?
2. How did you handle that situation?
3. Can you give an example of a time when you had to mediate a conflict within your team?
4. How did you facilitate open communication during the conflict?
5. How did you mediate discussions to find a collaborative solution?
6. How can this experience help you as a Certified nursing assistant?
7. How do you prioritize


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



1. Can you discuss the challenges of working with big data and techniques to handle scalability, performance, and resource constraints?
2. How do you ensure data quality and consistency when working with big data?
3. What are some common data integration challenges when working with big data?
4. Can you explain the concept of data partitioning and how it helps with big data processing?
5. How do you handle data privacy and security concerns when working with big data?
6.


,ground_truth_response,original_model_response,peft_model_response
0,790 why did you decide on a career...,\n1. Why did you choose to become a nursing as...,\n1. Can you tell me about your experience as ...
1,790 why did you decide on a career...,\n1. Can you tell me about a time when you had...,\n1. Can you tell me about a time when you had...
2,832 why would you be the best candidate t...,\nInterviewer: Can you tell me about a time wh...,\nInterviewer: Can you tell me about a time wh...
3,790 why did you decide on a career...,\n1. Can you tell me about a time when you had...,\n1. Can you tell me about a time when you had...
4,2127 What programming languages are you pro...,\n1. Can you discuss the challenges of working...,\n1. Can you discuss the challenges of working...


In [77]:
df.iloc[0]

original_model_response    \n1. Can you tell me about your experience as ...
peft_model_response        \n1. Why did you choose to become a nursing as...
Name: 0, dtype: object

In [71]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=c333764a4ef5ebbe3b66bf2ca8f34a7728f382a47e946ef764606600afcd1f90
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [91]:
import evaluate

rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_response,
    references=ground_truth_response[0:len(original_model_response)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_response,
    references=ground_truth_response[0:len(peft_model_response)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.20385520948369024, 'rouge2': 0.12328301886792455, 'rougeL': 0.171788411818597, 'rougeLsum': 0.1744837053002191}
PEFT MODEL:
{'rouge1': 0.38164692030318637, 'rouge2': 0.2662982871791761, 'rougeL': 0.3400653430626621, 'rougeLsum': 0.34043424152252316}


In [92]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 17.78%
rouge2: 14.30%
rougeL: 16.83%
rougeLsum: 16.60%
